In [65]:
#2
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [ ]:
import os
import requests
from dotenv import load_dotenv
from langgraph.graph import StateGraph

# 메뉴 데이터 파일 경로
MENU_FILE = "C:/mylangchain/langchain_basic/data/cafe_menu_data.txt"

# 메뉴 정보를 불러와 dict로 구성: {메뉴명: {재료, 설명, 가격}}
def load_menu_data(filepath):
    menu_dict = {}
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("|")
            if len(parts) != 4:
                print(f"⚠️ 잘못된 줄 건너뜀: {line.strip()}")
                continue
            name, ingredients, description, price = parts
            menu_dict[name] = {
                "ingredients": ingredients,
                "price": price
            }
    return menu_dict

menu_data = load_menu_data(MENU_FILE)

# 위키피디아 설명 가져오기 함수
def fetch_wikipedia_summary(query: str) -> str:
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data.get("extract", "위키 설명을 찾을 수 없습니다.")
    except Exception as e:
        return f"위키 정보 오류: {e}"

# 1. booq 노드
def booq_node(state: dict) -> dict:
    print("🤖 booq: 어떤 메뉴에 대해 알려드릴까요?")
    return state

# 2. 사용자 입력 노드
def user_message_node(state: dict) -> dict:
    user_input = "딸기 스무디에 대해서 설명 해줘"  # 입력 예시
    state["messages"].append(user_input)
    return state

# 3. 분류 노드
def classify_node(state: dict) -> dict:
    user_message = state["messages"][-1]
    if "추천" in user_message:
        state["type"] = "recommend"
    elif "가격" in user_message:
        state["type"] = "price"
    else:
        state["type"] = "menu"
    return state

# 4. 메뉴 정보 제공 노드
def info_node(state: dict) -> dict:
    user_message = state["messages"][-1]
    menu_name = user_message.split()[0]

    if menu_name in menu_data:
        ingredients = f"{menu_name} 재료: {menu_data[menu_name]['ingredients']}"
        summary = fetch_wikipedia_summary(menu_name)
        description = f"{menu_name} 설명 (위키): {summary}"
        state["messages"].append(ingredients)
        state["messages"].append(description)
    else:
        state["messages"].append(f"{menu_name}에 대한 정보를 찾을 수 없습니다.")
    return state

# 5. 참고 노드
def reference_node(state: dict) -> dict:
    ref = "📚 참고: 설명은 Wikipedia, 재료와 가격은 cafe_menu_data.txt 기반입니다."
    state["messages"].append(ref)
    return state

# 6. 가격 노드
def price_node(state: dict) -> dict:
    user_message = state["messages"][1]
    menu_name = user_message.split()[0]
    if menu_name in menu_data:
        price_info = f"🤖 카페봇: {menu_name} 가격은 ₩{menu_data[menu_name]['price']} 입니다."
        state["messages"].append(price_info)
    else:
        state["messages"].append(f"{menu_name}의 가격 정보를 찾을 수 없습니다.")
    return state

# 7. LangGraph 구성
graph = StateGraph(dict)
graph.add_node("booq", booq_node)
graph.add_node("user_message", user_message_node)
graph.add_node("classify", classify_node)
graph.add_node("info", info_node)
graph.add_node("reference", reference_node)
graph.add_node("price", price_node)

graph.set_entry_point("booq")
graph.add_edge("booq", "user_message")
graph.add_edge("user_message", "classify")
graph.add_edge("classify", "info")
graph.add_edge("info", "reference")
graph.add_edge("reference", "price")
graph.set_finish_point("price")

# 실행
cafe_graph = graph.compile()

init_state = {
    "messages": [],
    "type": None
}

final_state = cafe_graph.invoke(init_state)

# 출력
print("\n📢 전체 대화 흐름:")
for msg in final_state["messages"]:
    print(msg)
